In [1]:
import numpy as np

from ficar import FICAR

In [2]:
system = FICAR(n_classes=4, class_names=['male_w', 'female_w', 'male_b', 'female_b'])
system.train("dummyDL")

P(0,1)-> Fit with pair ('male_w', 'female_w')
P(0,2)-> Fit with pair ('male_w', 'male_b')
P(0,3)-> Fit with pair ('male_w', 'female_b')
P(1,2)-> Fit with pair ('female_w', 'male_b')
P(1,3)-> Fit with pair ('female_w', 'female_b')
P(2,3)-> Fit with pair ('male_b', 'female_b')


In [3]:
y_pred = system.predict('dummyIns')
print(system.decisions)
print(y_pred)

ARGMAX IN (0,3)
[[0.         0.51111708 0.13390356 0.90626084]
 [0.48888292 0.         0.59429718 0.45532822]
 [0.86609644 0.40570282 0.         0.84104569]
 [0.09373916 0.54467178 0.15895431 0.        ]]
female_b
